# Ejemplo de la técnica `stacking` usando el paquete `vecstack`
---

A partir de la técnica **stacking** se construyen múltiples modelos de distinto tipo y un modelo supervisor que es entrenado para combinar de la mejor forma posible las predicciones obtenidas a partir de los modelos primarios.

Como caso de uso se utilizará el dataset `iris` como simple dataset de clasificación multiclase.

# Importación de las librerias necesarias
---

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from vecstack import stacking

import warnings
warnings.filterwarnings('ignore')

# Lectura del set de datos
---

In [2]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [3]:
X = pd.DataFrame(X, columns=iris.feature_names)
y = pd.DataFrame(y, columns=['target'])
X.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
69,5.6,2.5,3.9,1.1
18,5.7,3.8,1.7,0.3
7,5.0,3.4,1.5,0.2
53,5.5,2.3,4.0,1.3
54,6.5,2.8,4.6,1.5


# Partición entrenamiento y test
---

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Definición de los modelos de primer nivel para aplicar la técnica stacking posteriormente
---

In [5]:
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3)
]

Los modelos presentan una serie de varlores en sus hiperparámetros. Habría que realizar un análisis previo de dicha búsqueda óptima.

## Predicciones para los modelos definidos

In [6]:
S_train, S_test = stacking(models, X_train, y_train, X_test, regression=False, mode='oof_pred_bag', needs_proba=False,
                           save_dir=None, metric=accuracy_score, n_folds=4, stratified=True, shuffle=True, 
                           random_state=0, verbose=2)

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [1.00000000]
    fold  1:  [0.92592593]
    fold  2:  [0.96153846]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.97186610] + [0.03082285]
    FULL:     [0.97142857]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96296296]
    fold  1:  [0.92592593]
    fold  2:  [0.92307692]
    fold  3:  [0.96000000]
    ----
    MEAN:     [0.94299145] + [0.01854705]
    FULL:     [0.94285714]

model  2:     [XGBClassifier]
    fold  0:  [0.96296296]
    fold  1:  [0.96296296]
    fold  2:  [0.92307692]
    fold  3:  [0.92000000]
    ----
    MEAN:     [0.94225071] + [0.02074080]
    FULL:     [0.94285714]



Para evitar el ajuste excesivo, la validación cruzada se suele utilizar para predecir la parte OOB (no disponible) del conjunto de entrenamiento. Es decir, al definir el número de pliegues el proceso es el siguiente:
- Dividimos nuestro set de datos en 4 pliegues.
- Para cada pliegue, tomamos los tres pliegues restantes y definimos nuestro modelo. La medida de evaluación en el conjunto train la hacemos realizando las predicciones sobre el pliegue tomado. 
- Posteriormente, realizamos las predicciones sobre el set de datos destinado para el test.
- Una vez que hemos hecho este proceso en cada pliege, obtenemos la media de la precisión alcanzada en cada uno de ellos para obtener una medida global del método.

La función tiene varias entradas:
- **models** : los modelos de primer nivel que definimos anteriormente
- **X_train, y_train, X_test** : nuestros datos
- **regression** : booleano que indica si queremos usar la función de regresión. En nuestro caso puesto en Falso ya que esta es una clasificación.
- **mode**: utilizando la descripción anterior de fuera de pliegue durante la validación cruzada
- **needs_proba** : booleano que indica si necesita las probabilidades de las etiquetas de clase
- **save_dir** : guarda el resultado en el directorio Boolean
- **metric** : qué métrica de evaluación usar (importamos la puntuación de precisión al principio)
- **n_folds** : cuántos pliegues usar para la validación cruzada
- **stratified** : si se debe utilizar la validación cruzada estratificada
- **shuffle** : si se barajan los datos
- **random_state** : establecer un estado aleatorio para reproducibilidad
- **verbose** : 2 aquí se refiere a imprimir toda la información

# Predicciones finales. Ajustar los modelos anteriores
---

Todo lo que queda por hacer es ajustar los modelos de segundo nivel de nuestra elección en nuestras predicciones para hacer nuestras predicciones finales. En nuestro caso, vamos a utilizar un clasificador XGBoost. Este paso no es significativamente diferente de un ajuste y predicción regular en sklearn, excepto por el hecho de que en lugar de usar X_train para entrenar a nuestro modelo, estamos usando nuestras predicciones S_train.

In [7]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Precisión alcanzada: %.8f' % accuracy_score(y_test, y_pred))

Precisión alcanzada: 0.97777778
